In [1]:
import geopandas as gpd
import pandas as pd

data_dir = r"D:\Landslide\data\raw\merge_landcover\gyeongnam_landcover.gpkg"
column='L2_NAME'

# 데이터 로드
gdf = gpd.read_file(data_dir)
print(f"데이터 형태: {gdf.shape}")
print(f"컬럼 목록: {list(gdf.columns)}")

# L2_NAME 컬럼의 고유값 확인
if column in gdf.columns:
    unique_values = gdf[column].unique()
    print(f"\n{column} 컬럼의 고유값 개수: {len(unique_values)}")
    print(f"\n{column} 컬럼의 고유값:")
    for i, value in enumerate(sorted(unique_values), 1):
        print(f"{i:2d}. {value}")
    
    # 각 고유값의 빈도수도 확인
    print(f"\n{column} 컬럼의 값별 빈도수:")
    value_counts = gdf[column].value_counts()
    print(value_counts)
else:
    print(f"'{column}' 컬럼이 데이터에 존재하지 않습니다.")

데이터 형태: (1430072, 12)
컬럼 목록: ['Id', 'L2_CODE', 'L2_NAME', 'IMG_NAME', 'IMG_DATE', 'ETC_INFO', 'UD_INFO', 'INX_NUM', 'layer', 'path', 'new_L2', 'geometry']

L2_NAME 컬럼의 고유값 개수: 22

L2_NAME 컬럼의 고유값:
 1. 공공시설지역
 2. 공업지역
 3. 과수원
 4. 교통지역
 5. 기타재배지
 6. 내륙수
 7. 내륙습지
 8. 논
 9. 문화·체육·휴양지역
10. 밭
11. 상업지역
12. 시설재배지
13. 연안습지
14. 인공나지
15. 인공초지
16. 자연나지
17. 자연초지
18. 주거지역
19. 침엽수림
20. 해양수
21. 혼효림
22. 활엽수림

L2_NAME 컬럼의 값별 빈도수:
L2_NAME
내륙습지          262398
교통지역          211368
인공초지          207801
활엽수림          101137
밭              94225
혼효림            88695
침엽수림           83577
내륙수            79945
인공나지           75334
논              55326
과수원            31099
주거지역           26058
공공시설지역         25122
자연나지           20197
상업지역           19610
시설재배지          16214
기타재배지           8423
공업지역            8119
연안습지            7477
해양수             2721
문화·체육·휴양지역      2666
자연초지            2560
Name: count, dtype: int64


In [2]:
import os

# 출력 디렉토리 설정
output_dir = r"D:\Landslide\data\processed\gyeongnam\landcover_classified"
os.makedirs(output_dir, exist_ok=True)

# L2_NAME을 6개 분류로 매핑
classification_mapping = {
    # forest(산림)
    'forest': ['침엽수림', '활엽수림', '혼효림'],
    
    # bareland(나지)
    'bareland': ['인공나지', '자연나지'],
    
    # agri(농경지)
    'agri': ['논', '밭', '과수원', '시설재배지', '기타재배지'],
    
    # urban(도시/개발지)
    'urban': ['주거지역', '상업지역', '공업지역', '공공시설지역', '교통지역', '문화·체육·휴양지역'],
    
    # grass(초지)
    'grass': ['인공초지', '자연초지'],
    
    # water_ratio(수역/습지)
    'water_ratio': ['내륙수', '해양수', '내륙습지', '연안습지']
}

print("분류별 L2_NAME 매핑:")
for category, l2_names in classification_mapping.items():
    print(f"\n{category}:")
    for name in l2_names:
        count = gdf[gdf[column] == name].shape[0] if name in gdf[column].values else 0
        print(f"  - {name}: {count:,}개")

# 각 분류별로 데이터 필터링 및 저장
for category, l2_names in classification_mapping.items():
    # 해당 분류에 속하는 데이터 필터링
    filtered_gdf = gdf[gdf[column].isin(l2_names)].copy()
    
    if len(filtered_gdf) > 0:
        # 새로운 분류 컬럼 추가
        filtered_gdf['classification'] = category
        
        # 파일 경로 설정
        output_path = os.path.join(output_dir, f"gyeongnam_{category}.gpkg")
        
        # GPKG 파일로 저장
        filtered_gdf.to_file(output_path, driver='GPKG')
        
        print(f"\n{category} 저장 완료:")
        print(f"  - 파일: {output_path}")
        print(f"  - 데이터 개수: {len(filtered_gdf):,}개")
        print(f"  - 포함된 L2_NAME: {', '.join(l2_names)}")
    else:
        print(f"\n{category}: 해당하는 데이터가 없습니다.")

print(f"\n모든 분류 파일이 {output_dir}에 저장되었습니다.")


분류별 L2_NAME 매핑:

forest:
  - 침엽수림: 83,577개
  - 활엽수림: 101,137개
  - 혼효림: 88,695개

bareland:
  - 인공나지: 75,334개
  - 자연나지: 20,197개

agri:
  - 논: 55,326개
  - 밭: 94,225개
  - 과수원: 31,099개
  - 시설재배지: 16,214개
  - 기타재배지: 8,423개

urban:
  - 주거지역: 26,058개
  - 상업지역: 19,610개
  - 공업지역: 8,119개
  - 공공시설지역: 25,122개
  - 교통지역: 211,368개
  - 문화·체육·휴양지역: 2,666개

grass:
  - 인공초지: 207,801개
  - 자연초지: 2,560개

water_ratio:
  - 내륙수: 79,945개
  - 해양수: 2,721개
  - 내륙습지: 262,398개
  - 연안습지: 7,477개

forest 저장 완료:
  - 파일: D:\Landslide\data\processed\gyeongnam\landcover_classified\gyeongnam_forest.gpkg
  - 데이터 개수: 273,409개
  - 포함된 L2_NAME: 침엽수림, 활엽수림, 혼효림

bareland 저장 완료:
  - 파일: D:\Landslide\data\processed\gyeongnam\landcover_classified\gyeongnam_bareland.gpkg
  - 데이터 개수: 95,531개
  - 포함된 L2_NAME: 인공나지, 자연나지

agri 저장 완료:
  - 파일: D:\Landslide\data\processed\gyeongnam\landcover_classified\gyeongnam_agri.gpkg
  - 데이터 개수: 205,287개
  - 포함된 L2_NAME: 논, 밭, 과수원, 시설재배지, 기타재배지

urban 저장 완료:
  - 파일: D:\Landslide\data\processed\gyeongn